In [2]:
# Importing necessary libraries here
import sys
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from gensim.models import Word2Vec
from gensim.models.phrases import Phrases, Phraser
from gensim.models import KeyedVectors
from gensim.models.word2vec import Word2Vec
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.tokenize import word_tokenize

import warnings 
warnings.filterwarnings('ignore') # no warnings

In [3]:
# Reading the data
df = pd.read_csv("comma_separated_ing.csv")

In [4]:
df.head()

,Unnamed: 0,Recipe_title_x,ingredient_name
0,0,Creamy Chicken Curry,"chicken breast, garlic, soy yogurt, turmeric, ..."
1,1,Leftover Turkey Biryani With Red Onion and C...,"sunflower oil, carrot, red onion, basmati rice..."
2,2,Seva Bean Vegetable Stew,"ghee, kidney bean, onion, garlic, carrot, pota..."
3,3,10 Second Mango Yogurt Smoothie,"mango nectar, yogurt"
4,4,17 Island Spicy Tomato Saffron Soup,"oil, ginger, onion, carrot, garlic, lemongrass..."


In [6]:
#Utitlity functions
def remove_comma(text):
     return text.replace(',','')
def remove_quote(text):
     return text.replace("'",'')
def remove_bracket1(text):
     return text.replace("]",'')
def remove_bracket2(text):
     return text.replace("[",'')
def remove_doublequote(text):
     return text.replace('"','')

df['cleaned'] = df['ingredient_name'].apply(remove_comma)
df['cleaned'] = df.cleaned.apply(remove_quote)
df['cleaned'] = df.cleaned.apply(remove_bracket1)
df['cleaned'] = df.cleaned.apply(remove_bracket2)
df['cleaned'] = df.cleaned.apply(remove_doublequote)

In [7]:
#splitting the description into words
corpus = []
for words in df['cleaned']:
    corpus.append(words.split())

In [9]:
#tag each document 
documents = [TaggedDocument(doc, [str(df['Recipe_title_x'][i])]) for i, doc in enumerate(corpus)]

In [10]:
max_epochs = 10
vec_size = 100
alpha = 0.025

model3 = Doc2Vec(vector_size=vec_size,
                alpha=alpha, 
                min_alpha=0.00025,
                min_count=1,
                dm =1,
                epochs=max_epochs)
  
model3.build_vocab(documents)

for epoch in range(max_epochs):
    print('iteration {0}'.format(epoch))
    model3.train(documents,
                total_examples=model3.corpus_count,
                epochs=model3.epochs)
    # decrease the learning rate
    model3.alpha -= 0.0002
    # fix the learning rate, no decay
    model3.min_alpha = model3.alpha

iteration 0
iteration 1
iteration 2
iteration 3
iteration 4
iteration 5
iteration 6
iteration 7
iteration 8
iteration 9


In [12]:
model3.wv.most_similar('potato',topn=10)
# model3.wv.most_similar('potatoes',topn=10)

[('chili', 0.34201884269714355),
 ('vegetable', 0.3401672840118408),
 ('soda', 0.3359622061252594),
 ('tomato', 0.3299388587474823),
 ('seed', 0.32142019271850586),
 ('powder', 0.3110930621623993),
 ('white', 0.3092752993106842),
 ('ghee', 0.2976037263870239),
 ('french', 0.29299691319465637),
 ('cashew', 0.29211923480033875)]

In [13]:
model3.save("ingredients_word2vec.model")